In [2]:
import torch
import torch.nn.functional as F
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering, AutoModelForTokenClassification,
    pipeline, AutoModel
)
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Using device: {device}")
torch.manual_seed(42)

PyTorch version: 2.9.0+cu128
CUDA available: True
Using device: cuda


In [4]:
# Sentiment Analysis
print("\n=== SENTIMENT ANALYSIS DEMO ===")
        
# Using pipeline (easiest way)
classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
        
sample_texts = [
    "I love this new product! It's amazing!",
    "This is terrible, worst experience ever.",
    "The weather is okay today, nothing special.",
    "Absolutely fantastic work on this project!",
    "I'm not sure how I feel about this change."
]
        
print("Sample texts and their sentiments:")
for text in sample_texts:
    result = classifier(text)
    print(f"Text: {text}")
    print(f"Sentiment: {result[0]['label']} (confidence: {result[0]['score']:.3f})")
    print("-" * 50)


=== SENTIMENT ANALYSIS DEMO ===


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Sample texts and their sentiments:
Text: I love this new product! It's amazing!
Sentiment: positive (confidence: 0.990)
--------------------------------------------------
Text: This is terrible, worst experience ever.
Sentiment: negative (confidence: 0.945)
--------------------------------------------------
Text: The weather is okay today, nothing special.
Sentiment: positive (confidence: 0.781)
--------------------------------------------------
Text: Absolutely fantastic work on this project!
Sentiment: positive (confidence: 0.980)
--------------------------------------------------
Text: I'm not sure how I feel about this change.
Sentiment: negative (confidence: 0.688)
--------------------------------------------------


In [6]:
# Question and Answer
print("\n=== QUESTION ANSWERING DEMO ===")
        
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
        
context = """
Hugging Face is a company that develops tools for building applications using 
machine learning. The company was founded in 2016 by Clément Delangue, Julien Chaumond, 
and Thomas Wolf. Hugging Face is known for its transformers library, which provides 
pre-trained models for natural language processing tasks. The company is headquartered 
in New York City and has raised significant funding from investors.
"""
        
questions = [
"When was Hugging Face founded?",
"Who are the founders of Hugging Face?",
"What is Hugging Face known for?",
"Where is Hugging Face headquartered?",
"Why Pakistan cricket team sucks?"
]
        
for question in questions:
    result = qa_pipeline(question=question, context=context)
    print(f"Question: {question}")
    print(f"Answer: {result['answer']}")
    print(f"Confidence: {result['score']:.3f}")
    print("-" * 50)

Device set to use cuda:0



=== QUESTION ANSWERING DEMO ===
Question: When was Hugging Face founded?
Answer: 2016
Confidence: 0.982
--------------------------------------------------
Question: Who are the founders of Hugging Face?
Answer: Clément Delangue, Julien Chaumond, 
and Thomas Wolf
Confidence: 0.950
--------------------------------------------------
Question: What is Hugging Face known for?
Answer: transformers library
Confidence: 0.685
--------------------------------------------------
Question: Where is Hugging Face headquartered?
Answer: New York City
Confidence: 0.996
--------------------------------------------------
Question: Why Pakistan cricket team sucks?
Answer: The company is headquartered 
in New York City and has raised significant funding from investors
Confidence: 0.020
--------------------------------------------------


In [7]:
# Entity Recognition
print("\n=== NAMED ENTITY RECOGNITION DEMO ===")
        
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english",aggregation_strategy="simple")
        
sample_texts = [
            "Apple Inc. was founded by Steve Jobs in Cupertino, California.",
            "The meeting with Microsoft will be held in Seattle next Tuesday.",
            "Barack Obama was the 44th President of the United States.",
            "Google's headquarters are located in Mountain View, California."
        ]
        
for text in sample_texts:
    entities = ner_pipeline(text)
    print(f"Text: {text}")
    print("Entities found:")
    for entity in entities:
        print(f"  - {entity['word']}: {entity['entity_group']} (confidence: {entity['score']:.3f})")
    print("-" * 50)


=== NAMED ENTITY RECOGNITION DEMO ===


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Text: Apple Inc. was founded by Steve Jobs in Cupertino, California.
Entities found:
  - Apple Inc: ORG (confidence: 1.000)
  - Steve Jobs: PER (confidence: 0.989)
  - Cupertino: LOC (confidence: 0.971)
  - California: LOC (confidence: 0.999)
--------------------------------------------------
Text: The meeting with Microsoft will be held in Seattle next Tuesday.
Entities found:
  - Microsoft: ORG (confidence: 1.000)
  - Seattle: LOC (confidence: 0.999)
--------------------------------------------------
Text: Barack Obama was the 44th President of the United States.
Entities found:
  - Barack Obama: PER (confidence: 0.999)
  - United States: LOC (confidence: 0.995)
--------------------------------------------------
Text: Google's headquarters are located in Mountain View, California.
Entities found:
  - Google: ORG (confidence: 0.999)
  - Mountain View: LOC (confidence: 0.995)
  - California: LOC (confidence: 0.998)
--------------------------------------------------


In [8]:
print("\n=== TEXT SUMMARIZATION DEMO ===")
        
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        
long_text = """
        Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to 
        the natural intelligence displayed by humans and animals. Leading AI textbooks define 
        the field as the study of "intelligent agents": any device that perceives its environment 
        and takes actions that maximize its chance of successfully achieving its goals. 
        Colloquially, the term "artificial intelligence" is often used to describe machines 
        that mimic "cognitive" functions that humans associate with the human mind, such as 
        "learning" and "problem solving". As machines become increasingly capable, tasks 
        considered to require "intelligence" are often removed from the definition of AI, 
        a phenomenon known as the AI effect. A quip in Tesler's Theorem says "AI is whatever 
        hasn't been done yet." For instance, optical character recognition is frequently 
        excluded from things considered to be AI, having become a routine technology. 
        Modern machine learning techniques are heavy on data and require substantial 
        computational power.
        """
        
summary = summarizer(long_text, max_length=100, min_length=30, do_sample=False)
print("\nSummary:")
print(summary[0]['summary_text'])


=== TEXT SUMMARIZATION DEMO ===


Device set to use cuda:0



Summary:
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents" As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI.


In [10]:
def text_generation_demo():
    """Demonstrate text generation"""
    print("\n=== TEXT GENERATION DEMO ===")
        
    generator = pipeline("text-generation", model="gpt2",max_length=100,num_return_sequences=2)
    prompts = [
            "The future of artificial intelligence is",
            "In the world of machine learning,",
            "Natural language processing helps us"
    ]
        
    for prompt in prompts:
        print(f"Prompt: {prompt}")
        generated_texts = generator(prompt)
        for i, text in enumerate(generated_texts):
            print(f"Generation {i+1}: {text['generated_text']}")
        print("-" * 50)
text_generation_demo()

Device set to use cuda:0



=== TEXT GENERATION DEMO ===


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: The future of artificial intelligence is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation 1: The future of artificial intelligence is going to be of big, beautiful, and exciting proportions."

Explore further: Robots will find smarter life
Generation 2: The future of artificial intelligence is very bright and tomorrow could usher in another epoch. In an interview with the New York Observer, Joseph Kroeber, the director of the Carnegie Institution for Science, described a future with "a world where people would spend more money buying the technologies that will make us happy and which will lead to greater prosperity."[42]

Kroeber warned of the impact of technological change on human society. The Industrial Revolution unleashed an unprecedented concentration of capital and resources needed for a society that began with the expansion of machinery and was still expanding at a rapid rate.

He pointed out that the rise of mass media and the increase in the Internet make technology the main driver of changing social life, and could lead us to create a more equitable, c

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation 1: In the world of machine learning, these results seem to have been particularly significant, at least in academia. In 2012, in an article in the Proceedings of the National Academy of Sciences, Professor Daniel Jadakas of the University of New Mexico conducted a study with 17 different researchers and found that their dataset was much more accurate.

Advertisement

In 2011, the Stanford Artificial Intelligence Lab's Jadakas et al published a long paper with an additional scientist looking for some interesting results:

A significant group of researchers from MIT, Yale and the University of Pittsburgh, with the support of the AI lab, have shown that when an intelligent neural network works successfully with a large set of data, there's an increased rate of activation in the neurons of the left and right, resulting in a higher level of creativity and self-awareness. In other words, this network can build up to more than a hundred million neurons, which are then trained on a 